In [1]:
from date.date_env import DateEnv

In [2]:
env = DateEnv()

In [3]:
env.reset() # reset environment to a new, random state
print(env.render())


(0, 0, 0)


In [4]:
env.reset()

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.sample_actions()
    state, reward, done = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'state': env.render(),
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 2
Penalties incurred: 0


In [5]:
frames

[{'state': (5, -1, 0), 'action': 5, 'reward': -1},
 {'state': (3, -2, 0), 'action': 3, 'reward': -3}]

In [6]:
actions = ['cars', 'sports', 'literature', 'history', 'machine learning and artificial intelligence', 'flirt', 'drink', 'leave']

In [7]:
action_lookup = {i: action for i, action in enumerate(actions)}

In [8]:
response_lookup = {
    -2: 'very bad',
    -1: 'bad',
    0: 'ambiguous',
    1: 'good',
    2: 'very good'
}


In [9]:
env.reset()

(0, 0, 0)

In [10]:
def parse_response(response):
    response_dict = {}
    (last_action, last_response, confidence), reward, done = response
    response_dict['last action'] = action_lookup[last_action]
    response_dict['last response'] = response_lookup[last_response]
    response_dict['self-confidence'] = confidence
    response_dict['reward'] = reward
    response_dict['done'] = done
    return response_dict

In [11]:
parse_response(env.step(5))

{'last action': 'flirt',
 'last response': 'ambiguous',
 'self-confidence': 0,
 'reward': 0,
 'done': False}

In [12]:
import numpy as np
q_table = np.zeros([env.n_actions, env.n_responses, env.date.me.max_confidence +1, env.n_actions])

In [38]:
#%%time
"""Training the agent"""

import random
#from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100000):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.sample_actions() # Explore action space
        else:
            action = np.argmax(q_table[state[0], state[1], state[2]]) # Exploit learned values

        next_state, reward, done = env.step(action)
        
        old_value = q_table[state[0], state[1], state[2], action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state[0], state[1], state[2], action] = new_value

        if reward == -10:
            penalties += 1
            
        
        state = next_state
        epochs += 1
        
    if i % 10000 == 0:
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 10000
Episode: 20000
Episode: 30000
Episode: 40000
Episode: 50000
Episode: 60000
Episode: 70000
Episode: 80000
Episode: 90000
Training finished.



In [39]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties, total_reward = 0, 0, 0
episodes = 1000

frames = []

for i in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    frame_set = []
    while not done:
        old_state = env.render()
        action = np.argmax(q_table[state])
        state, reward, done = env.step(action)

        if reward == -10:
            penalties += 1

        # Put each rendered frame into dict for animation
        frame_set.append({
            'state': old_state,
            'action': action,
            'reward': reward
            }
        )

        epochs += 1
    total_reward += sum([i['reward'] for i in frame_set])
    total_penalties += penalties
    total_epochs += epochs
    frames.append(frame_set)

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per episode: {total_reward / episodes}")

Results after 1000 episodes:
Average timesteps per episode: 5.763
Average penalties per episode: 0.0
Average reward per episode: -0.359


In [40]:
frames[4]

[{'state': (0, 0, 0), 'action': 3, 'reward': -1},
 {'state': (3, -1, 0), 'action': 6, 'reward': -1},
 {'state': (6, 0, 1), 'action': 4, 'reward': 0},
 {'state': (4, 0, 1), 'action': 2, 'reward': 1},
 {'state': (2, 1, 2), 'action': 2, 'reward': 1},
 {'state': (2, 1, 3), 'action': 5, 'reward': 1},
 {'state': (5, 1, 3), 'action': 5, 'reward': 12}]

In [16]:
action_lookup

{0: 'cars',
 1: 'sports',
 2: 'literature',
 3: 'history',
 4: 'machine learning and artificial intelligence',
 5: 'flirt',
 6: 'drink',
 7: 'leave'}